In [1]:
import cv2
import numpy as np 
import sys

In [2]:
np.set_printoptions(threshold=sys.maxsize)
np.seterr(divide='ignore', invalid='ignore')

{'divide': 'warn', 'over': 'warn', 'under': 'ignore', 'invalid': 'warn'}

Part 1a) There are multiple bright spots each for 1 face but the brightest spot is for the face in the template.When the kernel matches the image section, the ouput value will be a sum of squares, it would be positive and large. The normalisation takes care of the patches that would have high intensity so they do not give false bright spots. The sections with uniform intensity will go to zero and not give false bright spots. 

In [3]:
def findNoramlisedCorrelation(template, image):
    paddedImage = np.pad(image, ((template.shape[0] - 1, template.shape[0] - 1),
                                 (template.shape[1] - 1, template.shape[1] - 1)), 'constant')
#     print(paddedImage.shape, image.shape, template.shape)
#     cv2.imwrite("new.jpg", paddedImage)
    correlationMatrix = []
    for i in range(paddedImage.shape[0] - template.shape[0] + 1):
        tempList = []
        for j in range(paddedImage.shape[1] - template.shape[1] + 1):
            temp = paddedImage[i: i + template.shape[0], j: j + template.shape[1]]
            tempFlatten = temp.flatten()
            temp = temp - np.mean(tempFlatten)
            temp = temp / np.std(tempFlatten)
            temp = np.multiply(temp, template)
            temp = temp.flatten()
            tempList.append(np.sum(temp)/ (template.shape[0] * template.shape[1]))
        correlationMatrix.append(tempList)
    return correlationMatrix

In [4]:
groupPhoto = cv2.imread('gp.jpg')
groupPhoto = cv2.cvtColor(groupPhoto, cv2.COLOR_BGR2GRAY)
groupPhotoFace = cv2.imread('gpF.jpg')
groupPhotoFace = cv2.cvtColor(groupPhotoFace, cv2.COLOR_BGR2GRAY)
correlationMatrix = findNoramlisedCorrelation(groupPhotoFace, groupPhoto)
cv2.imwrite("1a.jpg", np.array(correlationMatrix))

True

part 1b) The template has 3 white lines with aproximately same intensity(high intensity as compared to other parts of the kernel). On cross corelation it will give high intensity where there is higher similarity with the kernel. The peak occurs at the original place in the image. The other lines are because the image has similar artifacts to the 3 lines. The diagonal edges are being sharpened.


In [5]:
template = cv2.imread('trailer.png')
template = cv2.cvtColor(template, cv2.COLOR_BGR2GRAY)
image = cv2.imread('u2cuba.jpg')
image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

cv2.imwrite("1b.jpg", np.array(findNoramlisedCorrelation(template, image)))

True

part1c) The results are very similar to the b part except the fact the peak brightness spot that is present in b is not present here. This happens because there is a scaling miss match in the template image and the section of image it was meant to correlate with. This shows that correlation is highly sensitive to any type of transformation and it is not practical to use it in real world scenarios where getting exact matching templates is not feasible.

In [6]:
template = cv2.imread('trailerSlightlyBigger.png')
template = cv2.cvtColor(template, cv2.COLOR_BGR2GRAY)
image = cv2.imread('u2cuba.jpg')
image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
cv2.imwrite("1c.jpg", np.array(findNoramlisedCorrelation(template, image)))

True

partd) Each pixel goes through O(mm) operations, each template has (nn) pixels so each template requires O(mmnn) operations. Hence for Ns sizes and Nr rotaions it will take O((Ns)(Nr)(mm)(nn)) operations.